In [1]:
% Numerical approximation to Poisson's equation over the square [a,b]x[a,b] with
% Dirichlet boundary conditions.  Uses a uniform mesh with (n+2)x(n+2) total
% points (i.e, n interior grid points). 
%
% Solves with classical multigrid (v-cycle and damped-Jacobi smoother).
%
% Input:
%     ffun : the RHS of poisson equation (i.e. the Laplacian of u).
%     gfun : the boundary function representing the Dirichlet B.C.
%      a,b : the interval defining the square
%        m : m+2 is the number of points in either direction of the mesh.
% Ouput:
%        u : the numerical solution of Poisson equation at the mesh points.
%      x,y : the uniform mesh
% Created By: Grady Wright
% Modified By: Andrew Jones
function [u,x,y] = fd2poissonmg(ffun,gfun,a,b,m)

k = round(log2(m+1));
if abs(log2(m+1)-k) > 100*eps
    error('fd2poissonMg:power2','m+1 must be a power of 2');
end

h = (b-a)/2^k;   % Mesh spacing

[x,y] = meshgrid(a:h:b);   % Uniform mesh, including boundary points.

idx = 2:m+1;
idy = 2:m+1;

% Compute boundary terms, south, north, east, west
ubs = feval(gfun,x(1,1:m+2),y(1,1:m+2));     % Include corners
ubn = feval(gfun,x(m+2,1:m+2),y(m+2,1:m+2)); % Include corners
ube = feval(gfun,x(idy,m+2),y(idy,m+2));     % No corners
ubw = feval(gfun,x(idy,1),y(idy,1));         % No corners

% Evaluate the RHS of Poisson's equation at the interior points.
f = feval(ffun,x,y);

% Inital guess
u = zeros(m+2,m+2);

% Add boundary terms to u
u(idx,1) = ubw; u(idx,m+2) = ube;
u(1,:) = ubs; u(m+2,:) = ubn;

tol = 1e-8;
r = inf;
nrmf = norm(f(:));
itr=0;
while norm(r(:)) > tol*nrmf
    [u,r] = vcycle(u,f,h,k);
    norm(r(:))
    itr+=1;
end
itr
end

function [u,r] = vcycle(u,f,h,k)

numSmooths = 3;
n = 2^k; n2 = n/2;
i = 2:n; i2 = 2:n2;

if k == 1
    % Only one interior point so solve the system:
    u(2,2) = 0.25*(u(1,2)+u(3,2)+u(2,1)+u(2,3)-h^2*f(2,2));
%    u
    printf("if \n")
    return;
else
    % Smooth the solution.
    printf("else \n");
    u = dampedJacobi(u,f,h,k,numSmooths);
    
%    u
    % Calculate residual
    r = zeros(n+1);
    r(i,i) = f(i,i) - (-4*u(i,i)+u(i-1,i)+u(i+1,i)+u(i,i-1)+u(i,i+1))/h^2;
%    r
    % Restrict the residual to the coarser grid using full-weighting
    jj = 3:2:n-1;
%    jj
    f2 = zeros(n2+1);
    f2(i2,i2) = (r(jj-1,jj)+r(jj+1,jj)+r(jj,jj-1)+r(jj,jj+1)+4*r(jj,jj))/8;
    f2
    % Repeat call to vcycle with the residual on coarser grid and with a
    % zero initial guess.
    u2 = vcycle(zeros(n2+1),f2,2*h,k-1);
    %fprintf("1")
    %display(size(u2))
%    u2
    % Interpolate (prolongate) the correction to the finer grid
    ut = zeros(n+1);
    %fprintf("2")
    ut(1:2:n+1,1:2:n+1) = u2;
%    ut
    ut(2:2:n,1:2:n+1) = 0.5*(u2(1:n2,:)+u2(2:n2+1,:));
    ut(:,2:2:n+1) = 0.5*(ut(:,1:2:n-1)+ut(:,3:2:n+1));
    
    % Update the solution and smooth again:
    u = dampedJacobi(u+ut,f,h,k,numSmooths);
    % Calculate residual
%    printf("%d \n",size(u));
    r(i,i) = f(i,i) - (-4*u(i,i)+u(i-1,i)+u(i+1,i)+u(i,i-1)+u(i,i+1))/h^2;
end
end

function u = dampedJacobi(u,f,h,k,numSmooths)
omega = 4/5;
n = 2^k;
i = 2:n;
f = h^2*f;
% Smooth u using underrelaxed Jacobi
for j=1:numSmooths
    u(i,i) = (0.25*omega)*(u(i-1,i)+u(i+1,i)+u(i,i-1)+u(i,i+1)-f(i,i)) + ...
                     (1-omega)*u(i,i);
end

end


In [2]:
% Script for testing fd2poisson over the square [a,b]x[a,b]

a = 0; 
b = 1;
k=3;
%errs=zeros(7,1);
%for k=4:10
%m = 2^k-1;  % Number of interior grid points in one direction
%%printf("k=%d",k);
%h = (b-a)/(m+1);
%
%% Laplacian(u) = f
%f = @(x,y) 10*pi^2*(1+cos(4*pi*(x+2*y))-2*sin(2*pi*(x+2*y))).*exp(sin(2*pi*(x+2*y)));  
%% u = g on Boundary
%g = @(x,y) exp(sin(2*pi*(x+2*y)));            
%
%% Exact solution is g.
%uexact = @(x,y) g(x,y);                     
%
%% Compute and time the solution
%%tic
%%[u,x,y] = fd2poisson(f,g,a,b,m);
%%gedirect = toc;
%
%%k
%[u,x,y] = fd2poissonmg(f,g,a,b,m);
%relerr=norm(u-uexact(x,y),2)/norm(uexact(x,y),2);
%errs(k-3,1)=relerr;
%end
m = 2^k-1;  % Number of interior grid points in one direction
%printf("k=%d",k);
h = (b-a)/(m+1);

% Laplacian(u) = f
f = @(x,y) 10*pi^2*(1+cos(4*pi*(x+2*y))-2*sin(2*pi*(x+2*y))).*exp(sin(2*pi*(x+2*y)));  
% u = g on Boundary
g = @(x,y) exp(sin(2*pi*(x+2*y)));            

[u,x,y] = fd2poissonmg(f,g,a,b,m);
%tic
%[u,x,y] =fd2poissondst(f,g,a,b,m);
%gedirect = toc;

%fprintf('Direct Gaussian elimination takes %d s\n',gedirect);

%% Plot solution
%figure, set(gcf,'DefaultAxesFontSize',10,'PaperPosition', [0 0 3.5 3.5]), 
%surf(x,y,u), xlabel('x'), ylabel('y'), zlabel('u(x,y)'),
%title(strcat('Numerical Solution to Poisson Equation, h=',num2str(h)));
%
%% Plot error
%figure, set(gcf,'DefaultAxesFontSize',10,'PaperPosition', [0 0 3.5 3.5]), 
%surf(x,y,u-uexact(x,y)),xlabel('x'),ylabel('y'), zlabel('Error'), 
%title(strcat('Error, h=',num2str(h)));

else 
f2 =

    0.00000    0.00000    0.00000    0.00000    0.00000
    0.00000   -7.28102  -22.13415  -47.91672    0.00000
    0.00000  -36.63607   -5.95357   -0.20616    0.00000
    0.00000   -7.28102  -22.13415  -47.91672    0.00000
    0.00000    0.00000    0.00000    0.00000    0.00000

else 
f2 =

    0.00000    0.00000    0.00000
    0.00000  -11.24636    0.00000
    0.00000    0.00000    0.00000

if 
ans =  25.484
else 
f2 =

   0.00000   0.00000   0.00000   0.00000   0.00000
   0.00000  -1.60952  -1.82790  -2.56067   0.00000
   0.00000  -2.00505  -1.19247  -1.86457   0.00000
   0.00000  -1.60952  -1.82790  -2.56067   0.00000
   0.00000   0.00000   0.00000   0.00000   0.00000

else 
f2 =

   0.00000   0.00000   0.00000
   0.00000  -1.01739   0.00000
   0.00000   0.00000   0.00000

if 
ans =  1.4445
else 
f2 =

   0.00000   0.00000   0.00000   0.00000   0.00000
   0.00000  -0.12712  -0.16512  -0.16808   0.00000
   0.00000  -0.15793  -0.17555  -0.17672   0.00000
   0.00000  -0.12